In [1]:
from dash import Dash, html, dcc, Input, Output
import yfinance as yf
import plotly.graph_objects as go
import pandas as pd
from plotly.subplots import make_subplots
import datetime


In [2]:
ticker = 'AAPL'
interval_val = '5m'
period_val = '60d'
aapl = yf.Ticker(ticker)
stock_data = aapl.history(period=period_val, interval=interval_val)
stock_data['price_difference'] = stock_data['Close'] - stock_data['Open']
stock_data['direction_volume'] = stock_data.apply(lambda row: -row['Volume'] if row['price_difference'] < 0 else row['Volume'], axis=1)
stock_data['cumulative_volume'] = stock_data['direction_volume'].cumsum()

In [3]:
stock_data.iloc[0:10]

,Open,High,Low,Close,Volume,Dividends,Stock Splits,price_difference,direction_volume,cumulative_volume
Datetime,,,,,,,,,,
2023-12-01 09:30:00-05:00,190.330002,190.490005,189.440002,189.500000,1932144,0.0,0.0,-0.830002,-1932144.0,-1932144.0
2023-12-01 09:35:00-05:00,189.509995,189.679504,189.229996,189.660004,710113,0.0,0.0,0.150009,710113.0,-1222031.0
2023-12-01 09:40:00-05:00,189.657303,189.899994,189.585007,189.774994,517361,0.0,0.0,0.117691,517361.0,-704670.0
2023-12-01 09:45:00-05:00,189.779999,190.039993,189.744995,189.899994,649366,0.0,0.0,0.119995,649366.0,-55304.0
2023-12-01 09:50:00-05:00,189.904999,190.240005,189.889999,190.175003,740010,0.0,0.0,0.270004,740010.0,684706.0
2023-12-01 09:55:00-05:00,190.179993,190.210007,189.910004,189.960007,453024,0.0,0.0,-0.219986,-453024.0,231682.0
2023-12-01 10:00:00-05:00,189.970001,190.500000,189.929993,190.320007,987842,0.0,0.0,0.350006,987842.0,1219524.0
2023-12-01 10:05:00-05:00,190.330002,190.440002,190.080002,190.214996,513289,0.0,0.0,-0.115005,-513289.0,706235.0
2023-12-01 10:10:00-05:00,190.214996,190.410004,190.119995,190.220001,551638,0.0,0.0,0.005005,551638.0,1257873.0


In [6]:
stock_data.size

46800

In [16]:
stock_data.iloc[len(stock_data)-1]

Open                 1.809950e+02
High                 1.814700e+02
Low                  1.809900e+02
Close                1.814200e+02
Volume               2.089637e+06
Dividends            0.000000e+00
Stock Splits         0.000000e+00
price_difference     4.250031e-01
direction_volume     2.089637e+06
cumulative_volume    1.571687e+08
Name: 2024-02-28 15:55:00-05:00, dtype: float64

In [4]:
stock_data.iloc[0:10]['cumulative_volume']

Datetime
2023-12-01 09:30:00-05:00   -1932144.0
2023-12-01 09:35:00-05:00   -1222031.0
2023-12-01 09:40:00-05:00    -704670.0
2023-12-01 09:45:00-05:00     -55304.0
2023-12-01 09:50:00-05:00     684706.0
2023-12-01 09:55:00-05:00     231682.0
2023-12-01 10:00:00-05:00    1219524.0
2023-12-01 10:05:00-05:00     706235.0
2023-12-01 10:10:00-05:00    1257873.0
2023-12-01 10:15:00-05:00    1735796.0
Name: cumulative_volume, dtype: float64

In [23]:
# Function to fetch data and calculate points of interest
def load_data(ticker,period_val,interval_val):
    aapl = yf.Ticker(ticker)
    stock_data = aapl.history(period=period_val, interval=interval_val)
    stock_data['price_difference'] = stock_data['Close'] - stock_data['Open']
    stock_data['direction_volume'] = stock_data.apply(lambda row: -row['Volume'] if row['price_difference'] < 0 else row['Volume'], axis=1)
    stock_data['cumulative_volume'] = stock_data['direction_volume'].cumsum()

    return stock_data

selected_ticker = 'AAPL'

period = '60d'
interval = '5m'

temp_data = load_data(selected_ticker,period,interval)
num_days = 60
current_time = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=-5)))
start_date = current_time  - datetime.timedelta(days=num_days)
end_date = current_time

In [25]:
current_time = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=-5)))  # Adjust UTC offset for "America/New_York"
print(current_time)
#value=[0, num_days-1]

2024-02-27 19:24:56.761210-05:00


In [22]:
print(temp_data)

None


In [24]:
temp_data.loc[start_date:end_date]

,Open,High,Low,Close,Volume,Dividends,Stock Splits,price_difference,direction_volume,cumulative_volume
Datetime,,,,,,,,,,
2024-01-02 09:30:00-05:00,187.149994,188.440002,186.350006,187.729996,7856580,0.0,0.0,0.580002,7856580.0,47964546.0
2024-01-02 09:35:00-05:00,187.759995,188.070007,187.130005,187.529999,1827415,0.0,0.0,-0.229996,-1827415.0,46137131.0
2024-01-02 09:40:00-05:00,187.539993,187.559998,186.059998,186.259995,2191393,0.0,0.0,-1.279999,-2191393.0,43945738.0
2024-01-02 09:45:00-05:00,186.250000,186.940002,186.009995,186.100006,1732940,0.0,0.0,-0.149994,-1732940.0,42212798.0
2024-01-02 09:50:00-05:00,186.089996,186.679993,186.089996,186.279999,1487644,0.0,0.0,0.190002,1487644.0,43700442.0
...,...,...,...,...,...,...,...,...,...,...
2024-02-27 15:35:00-05:00,182.320007,182.509995,182.279999,182.490005,626414,0.0,0.0,0.169998,626414.0,165280816.0
2024-02-27 15:40:00-05:00,182.490005,182.529999,182.289993,182.505493,581281,0.0,0.0,0.015488,581281.0,165862097.0
2024-02-27 15:45:00-05:00,182.500107,182.669998,182.460007,182.604996,855724,0.0,0.0,0.104889,855724.0,166717821.0


In [17]:


# Convert datetime index to date format
stock_data.index = stock_data.index.date

# Get the number of distinct dates
num_distinct_dates = stock_data.index.nunique()

print(num_distinct_dates)

60


In [15]:
stock_data.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,price_difference,direction_volume,cumulative_volume
Datetime,,,,,,,,,,
2023-11-30 09:30:00-05:00,189.839996,190.320007,189.619995,189.940002,2362725,0.0,0.0,0.100006,2362725.0,2362725.0
2023-11-30 09:35:00-05:00,189.949997,189.970001,189.371002,189.380005,664062,0.0,0.0,-0.569992,-664062.0,1698663.0
2023-11-30 09:40:00-05:00,189.377899,189.550003,189.149994,189.529800,706590,0.0,0.0,0.151901,706590.0,2405253.0
2023-11-30 09:45:00-05:00,189.520004,189.559998,189.050003,189.164993,629649,0.0,0.0,-0.355011,-629649.0,1775604.0
2023-11-30 09:50:00-05:00,189.169998,189.449997,189.120102,189.184998,563943,0.0,0.0,0.014999,563943.0,2339547.0


In [14]:
len(stock_data)

10

In [2]:
ticker = 'aapl'
aapl = yf.Ticker(ticker)
stock_data = aapl.history(period='1y', interval='60m')
calculated_values = []

# Your existing calculations here

In [3]:
stock_data.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2023-02-24 09:30:00-05:00,147.110001,147.190002,146.119995,146.289993,13191997,0.0,0.0
2023-02-24 10:30:00-05:00,146.289993,146.580002,145.729996,146.389999,7016244,0.0,0.0
2023-02-24 11:30:00-05:00,146.399994,146.899994,146.059998,146.251007,4694128,0.0,0.0
2023-02-24 12:30:00-05:00,146.259995,146.438202,145.720200,146.214996,4256702,0.0,0.0
2023-02-24 13:30:00-05:00,146.220001,146.369995,145.889999,146.110001,4236554,0.0,0.0


In [5]:
print(stock_data['Volume'].sum())

11690013868


In [10]:
stock_data['price_difference'] = stock_data['Close'] - stock_data['Open']
stock_data['direction_volume'] = stock_data.apply(lambda row: -row['Volume'] if row['price_difference'] < 0 else row['Volume'], axis=1)
stock_data['cumulative_volume'] = stock_data['direction_volume'].cumsum()

In [11]:
print(stock_data['cumulative_volume'])

Datetime
2023-02-24 09:30:00-05:00    -13191997.0
2023-02-24 10:30:00-05:00     -6175753.0
2023-02-24 11:30:00-05:00    -10869881.0
2023-02-24 12:30:00-05:00    -15126583.0
2023-02-24 13:30:00-05:00    -19363137.0
                                ...     
2024-02-23 11:30:00-05:00    963392595.0
2024-02-23 12:30:00-05:00    970298268.0
2024-02-23 13:30:00-05:00    967342384.0
2024-02-23 14:30:00-05:00    971370972.0
2024-02-23 15:30:00-05:00    966018078.0
Name: cumulative_volume, Length: 1749, dtype: float64


In [4]:
stock_data.index.dtype

datetime64[ns, America/New_York]

In [6]:
# Get current time in the "America/New_York" timezone
current_time = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=-5)))  # Adjust UTC offset for "America/New_York"

# Calculate start and end dates
end_date = current_time
start_date = end_date - datetime.timedelta(days=30)

# Filter stock_data by datetime index
filtered_stock_data = stock_data.loc[start_date:end_date]

print(end_date)
print(start_date)

print(filtered_stock_data)

2024-02-24 16:32:41.813977-05:00
2024-01-25 16:32:41.813977-05:00
                                 Open        High         Low       Close  \
Datetime                                                                    
2024-01-26 09:30:00-05:00  194.270004  194.759995  193.750000  194.684998   
2024-01-26 10:30:00-05:00  194.690002  194.744995  194.360001  194.389999   
2024-01-26 11:30:00-05:00  194.399994  194.475006  193.600006  193.720001   
2024-01-26 12:30:00-05:00  193.700104  193.710007  192.270004  192.660004   
2024-01-26 13:30:00-05:00  192.660004  192.729996  192.270004  192.475098   
...                               ...         ...         ...         ...   
2024-02-23 11:30:00-05:00  183.054993  183.115005  182.390106  182.559998   
2024-02-23 12:30:00-05:00  182.554993  182.960007  182.309998  182.955002   
2024-02-23 13:30:00-05:00  182.955002  182.970001  182.430298  182.470001   
2024-02-23 14:30:00-05:00  182.479996  182.695007  182.440002  182.585007   
2024-02-23